In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
print('hello')

hello


In [5]:
import quandl
quandl.ApiConfig.api_key = 'VDCe-Sck1jKwogtfPBtN'
quandl.ApiConfig.verify_ssl = False

In [15]:
#data = quandl.get_table('WIKI', ticker='AAPL')

In [14]:
data.head()

,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,stock_based_compsn_qd,cash_flow_oper_activity_qd,net_change_prop_plant_equip_qd,comm_stock_div_paid_qd,pref_stock_div_paid_qd,tot_comm_pref_stock_div_qd,wavg_shares_out,wavg_shares_out_diluted,eps_basic_net,eps_diluted_net
None,,,,,,,,,,,,,,,,,,,,,
0,AAPL,AAPL,Apple,Apple Inc.,NASDAQ,USD,2018-09-30,A,None,2018,...,NaN,NaN,NaN,NaN,None,NaN,19821.51,20000.44,3.0000,2.9800
1,AAPL,AAPL,Apple,Apple Inc.,NASDAQ,USD,2018-03-31,Q,None,2018,...,1348.0,15130.0,-4195.0,-3190.0,None,-3190.0,20099.51,20273.97,0.6875,0.6825
2,AAPL,AAPL,Apple,Apple Inc.,NASDAQ,USD,2018-06-30,Q,None,2018,...,1351.0,14488.0,-3267.0,-3653.0,None,-3653.0,19528.67,19706.44,0.5900,0.5850
3,AAPL,AAPL,Apple,Apple Inc.,NASDAQ,USD,2018-09-30,Q,None,2018,...,1345.0,19523.0,-3041.0,-3530.0,None,-3530.0,19821.51,20000.44,0.7425,0.7400
4,AAPL,AAPL,Apple,Apple Inc.,NASDAQ,USD,2018-12-31,Q,None,2019,...,1559.0,26690.0,-3355.0,-3568.0,None,-3568.0,18943.28,19093.01,1.0550,1.0450


In [16]:
import yfinance as yf

In [18]:
msft = yf.Ticker('MSFT')

In [19]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 181000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. Its Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certifica

In [21]:
from alpha_vantage.timeseries import TimeSeries
import time

In [22]:
api_key = '7DOY154L31ZDC2PG'

In [27]:
ts = TimeSeries(key=api_key, output_format='pandas')
data, meta_data = ts.get_daily_adjusted(symbol='AAPL', outputsize='full')

In [29]:
data.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2021-11-10,150.02,150.130,147.8500,147.92,147.920000,64351793.0,0.00,1.0
2021-11-09,150.20,151.428,150.0601,150.81,150.810000,56573449.0,0.00,1.0
2021-11-08,151.41,151.570,150.1600,150.44,150.440000,55020868.0,0.00,1.0
2021-11-05,151.89,152.200,150.0600,151.28,151.280000,65463883.0,0.22,1.0
2021-11-04,151.58,152.430,150.6400,150.96,150.740784,60394616.0,0.00,1.0


In [30]:
data.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
1999-11-05,84.62,88.37,84.00,88.31,0.675935,3721500.0,0.0,1.0
1999-11-04,82.06,85.37,80.62,83.62,0.640037,3384700.0,0.0,1.0
1999-11-03,81.62,83.25,81.00,81.50,0.623810,2932700.0,0.0,1.0
1999-11-02,78.00,81.69,77.31,80.25,0.614243,3564600.0,0.0,1.0
1999-11-01,80.00,80.69,77.37,77.62,0.594112,2487300.0,0.0,1.0
